In [27]:
from Bio.PDB import *
import pandas as pd
import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)
import csv
import numpy as np
from Bio.PDB.Polypeptide import three_to_one, one_to_three                           
import math
import requests as r
from Bio import SeqIO
from io import StringIO

In [28]:
def load_file(fName):
    df = pd.read_csv(fName)
    return df

In [29]:
def load_pd(fName):
    df = pd.read_csv(fName)
    df = df.dropna(subset = ['Affinity_mut_parsed', 'Affinity_wt_parsed', 'Temperature'])
    return df

skepiIn = "C:/Users/amber/Study_Documents/intern/inno/ppi with binding affinity/skempi/skempi_v2.csv"
ppi = load_pd(skepiIn)
print("Size of the data:", ppi.shape)


Size of the data: (6794, 29)


# wild-type library

## get dG data

In [30]:
dg_info = []
wild_list = []

cnt_single_chain = cnt_both_chain = 0

f1 = open("./skempi/wild.seq.txt", "w")
f2 = open("./skempi/wild.dg.txt", "w")
#print(ppi.shape)

for i in range(6794):
    data = ppi.iloc[i]
    complex_id = data[0]
    if complex_id not in wild_list:
        ent_name = data[0][:4]
    #     .lower()
        #print(ent_name)
        chains = data[0].split('_')[1:]
        #print(chains)
        assert(len(chains) == 2)
    #     print(i, len(ppi))
        chain_ID0 = chains[0]
        chain_ID1 = chains[1]

        # there also exists cases the two chains in one side
        if len(chain_ID0) == 1 and len(chain_ID1) == 1:
            kd = data['Affinity_wt_parsed']
            #print(kd)
            temp = data['Temperature'].strip()[:3]
            #print(temp)
            
            #get_dg(ent_name, chain_ID0, chain_ID1, kd, dg_info)
            wild_list.append(complex_id)

print(len(wild_list))

#for elem in dg_info:
    #print(elem, file=open("./skempi/wild.dg.txt", "a"))
  

225


there are 225 unique wild-type protein complex with two chains

## get sequencing data

### get seq from PDB bank

download the pdb file

In [31]:
for item in wild_list:
    ent_name = item[:4]
    print(ent_name)
    pdbl = PDBList()
    assert(len(ent_name) ==4)
    pdbl.retrieve_pdb_file(ent_name, pdir = './skempi/PDBs/', file_format='pdb')

1CSE
Structure exists: './skempi/PDBs/pdb1cse.ent' 
1ACB
Structure exists: './skempi/PDBs/pdb1acb.ent' 
1SBN
Structure exists: './skempi/PDBs/pdb1sbn.ent' 
1SIB
Structure exists: './skempi/PDBs/pdb1sib.ent' 
1TM1
Structure exists: './skempi/PDBs/pdb1tm1.ent' 
1Y1K
Structure exists: './skempi/PDBs/pdb1y1k.ent' 
1Y33
Structure exists: './skempi/PDBs/pdb1y33.ent' 
1Y34
Structure exists: './skempi/PDBs/pdb1y34.ent' 
1Y3B
Structure exists: './skempi/PDBs/pdb1y3b.ent' 
1Y4A
Structure exists: './skempi/PDBs/pdb1y4a.ent' 
1Y3C
Structure exists: './skempi/PDBs/pdb1y3c.ent' 
1Y48
Structure exists: './skempi/PDBs/pdb1y48.ent' 
1Y3D
Structure exists: './skempi/PDBs/pdb1y3d.ent' 
1TM4
Structure exists: './skempi/PDBs/pdb1tm4.ent' 
1TM5
Structure exists: './skempi/PDBs/pdb1tm5.ent' 
1TM3
Structure exists: './skempi/PDBs/pdb1tm3.ent' 
1TM7
Structure exists: './skempi/PDBs/pdb1tm7.ent' 
1TMG
Structure exists: './skempi/PDBs/pdb1tmg.ent' 
1TO1
Structure exists: './skempi/PDBs/pdb1to1.ent' 
2SIC
Structu

In [32]:
def get_seq(ent_name, chain, seqinfo):
#     print(ent_names, chain_ID0, chain_ID1)
    parser = PDBParser()
    structure = parser.get_structure(ent_name, './skempi/PDBs/pdb'+ ent_name.lower() + '.ent')
    #     assert(len(structure) == 1)
    model = structure[0]

    w_name = ent_name+'_'+ chain
    if w_name not in seqinfo:
        w_seq = ''

        chain = model[chain]
        ppb = PPBuilder()
        for pp in ppb.build_peptides(chain):
            w_seq += pp.get_sequence()
        #print(type(w_seq))
        sequence = w_name + " " + str(w_seq)
        seqinfo.append(sequence)
        
    return seqinfo

In [33]:
seqinfo = []

for item in wild_list:
    ent_name = item[:4]
    chain0 = item[-1]
    get_seq(ent_name, chain0, seqinfo)
    chain1 = item[-3]
    get_seq(ent_name, chain1, seqinfo)
    

In [34]:
len(seqinfo)

450

In [35]:
for elem in seqinfo:
    print(elem, file=open("./skempi/wild2.seq.txt", "a"))

# mutation sequence

In [36]:
mutated_dg_info= []
mutated_seq_list= []

f3 = open("./skempi/mut.seq.txt", "w")
#f4 = open("./skempi/mut.dg.txt", "w")
#print(ppi.shape)

for i in range(6794):
    data = ppi.iloc[i]
    complex_id = data[0]
    ent_name = data[0][:4]
    chains = data[0].split('_')[1:]
    #print(chains)
    assert(len(chains) == 2)
    #     print(i, len(ppi))
    chain_ID0 = chains[0]
    chain_ID1 = chains[1]

    # there also exists cases the two chains in one side
    if len(chain_ID0) == 1 and len(chain_ID1) == 1:
        kd = data['Affinity_mut_parsed']
        #print(kd)
        temp = data['Temperature'].strip()[:3]
        #print(temp)
        mutation_site = data['Mutation(s)_cleaned']
        mutated_chain_ID0 = chain_ID0 + "_" + mutation_site
        mutated_chain_ID1 = chain_ID1 + "_" + mutation_site
        #get_dg(ent_name, mutated_chain_ID0, mutated_chain_ID1, kd, mutated_dg_info)
        mutated_seq_list.append(ent_name + "_" + mutated_chain_ID0)
        mutated_seq_list.append(ent_name + "_" + mutated_chain_ID1)

#for elem in mutated_dg_info:
    #print(elem, file=open("./skempi/mut.dg.txt", "a"))

f3.close()
#f4.close()

There is 4165 mutated complex pair with two chains

In [37]:
len(mutated_seq_list)
mutated_seq_list[40:50]

['1TM1_E_TI39D',
 '1TM1_I_TI39D',
 '1TM1_E_TI39A',
 '1TM1_I_TI39A',
 '1TM1_E_EI41A',
 '1TM1_I_EI41A',
 '1TM1_E_TI39A,EI41A',
 '1TM1_I_TI39A,EI41A',
 '1TM1_E_TI39D,EI41A',
 '1TM1_I_TI39D,EI41A']

## sequencing data

read wild-type txt into dic

In [38]:
seq_wild = {}
with open("./skempi/wild2.seq.txt") as f:
    for line in f:
       (name, sequence) = line.split()
       seq_wild[name] = sequence


In [39]:
def mutation_on_seq(los, seq_mutated):
    info = los.split("_")
    ent_name = info[0]
    chain_id = info[1]
    ent_id = "_".join(info[:2])
    #print(ent_id)
    lo_mutation = info[2].split(",")
    #print(lo_mutation)
    seq_w = seq_wild[ent_id]

    for item in lo_mutation:
        mutation_chain = item[1]
        mutation_pos = int(item[2:-1])
        #print(mutation_pos)
        r_w = item[0]
        #print(r_w)
        r_m = item[-1]
        #print(r_m)
        
        if chain_id == mutation_chain:
            #print(seq_w)
            seq = list(seq_w)
            #print(seq[mutation_pos - 1])
            if mutation_pos > len(seq_w):
                errormeg = "exceed length in " + los
                print(errormeg)
            elif seq[mutation_pos -1] == r_w:
                seq[mutation_pos - 1] = r_m
                seq_m = "".join(seq)
                #print(seq_m)
                seq_mutated[los] = seq_m
                seq_w = seq_m
            else:
                errormeg = "not corresponding in " + los
                print(errormeg)
        else:
            seq_mutated[los] = seq_w



#### tests

In [40]:
seq_mutated = {}
mutation_on_seq('1TM1_I_TI39D,EI41A',seq_mutated)
print(seq_wild['1TM1_I'])
print(seq_mutated['1TM1_I_TI39D,EI41A'])

MKTEWPELVGKSVEEAKKVILQDKPAAQIIVLPVGTIVTMEYRIDRVRLFVDRLDNIAQVPRVG
MKTEWPELVGKSVEEAKKVILQDKPAAQIIVLPVGTIVDMAYRIDRVRLFVDRLDNIAQVPRVG


In [41]:
mutation_on_seq('1TM1_I_RI48C',seq_mutated)
print(seq_wild['1TM1_I'])
print(seq_mutated['1TM1_I_RI48C'])

MKTEWPELVGKSVEEAKKVILQDKPAAQIIVLPVGTIVTMEYRIDRVRLFVDRLDNIAQVPRVG
MKTEWPELVGKSVEEAKKVILQDKPAAQIIVLPVGTIVTMEYRIDRVCLFVDRLDNIAQVPRVG


In [42]:
mutation_on_seq('1TM1_E_EI41A',seq_mutated)
print(seq_wild['1TM1_E'])
print(seq_mutated['1TM1_E_EI41A'])
seq_wild['1TM1_E'] == seq_mutated['1TM1_E_EI41A']

AQSVPYGVSQIKAPALHSQGYTGSNVKVAVIDSGIDSSHPDLKVAGGASMVPSETNPFQDNNSHGTHVAGTVAALNNSIGVLGVAPSASLYAVKVLGADGSGQYSWIINGIEWAIANNMDVINMSLGGPSGSAALKAAVDKAVASGVVVVAAAGNEGTSGSSSTVGYPGKYPSVIAVGAVDSSNQRASFSSVGPELDVMAPGVSIQSTLPGNKYGAYNGTSMASPHVAGAAALILSKHPNWTNTQVRSSLENTTTKLGDSFYYGKGLINVQAAAQHHHHHH
AQSVPYGVSQIKAPALHSQGYTGSNVKVAVIDSGIDSSHPDLKVAGGASMVPSETNPFQDNNSHGTHVAGTVAALNNSIGVLGVAPSASLYAVKVLGADGSGQYSWIINGIEWAIANNMDVINMSLGGPSGSAALKAAVDKAVASGVVVVAAAGNEGTSGSSSTVGYPGKYPSVIAVGAVDSSNQRASFSSVGPELDVMAPGVSIQSTLPGNKYGAYNGTSMASPHVAGAAALILSKHPNWTNTQVRSSLENTTTKLGDSFYYGKGLINVQAAAQHHHHHH


True

In [43]:
mutation_on_seq('2WPT_A_FB79A,NA31A',seq_mutated)
print(seq_wild['2WPT_A'])
print(seq_mutated['2WPT_A_FB79A,NA31A'])

mutation_on_seq('2WPT_B_FB79A,NA31A',seq_mutated)
print(seq_wild['2WPT_B'])
print(seq_mutated['2WPT_B_FB79A,NA31A'])

KHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEHPDGSDLIYYPRDDREDSPEGIVKEIKEWRAANGKSGFKQ
KHSISDYTEAEFLEFVKKIARAEGATECDDAKLVREFERLTEHPDGSDLIYYPRDDREDSPEGIVKEIKEWRAANGKSGFKQ
PGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKEFKSFDDFRKAVWEEVSKDPELSKNLNPSNKSSVSKGYSPFTPKNQQVGCRKVYELHHDKGEVYDMDNIRVTTPK
PGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKEFKSFDDFRKAVWEEVSKDPELSKNLNPSNKSSVSKGYSPATPKNQQVGCRKVYELHHDKGEVYDMDNIRVTTPK


#### result

In [44]:
seq_mutated = {}
for item in mutated_seq_list:
    mutation_on_seq(item, seq_mutated)

not corresponding in 1S1Q_A_WA72A
not corresponding in 1S1Q_A_FA85A
exceed length in 1KBH_B_YB1172W,IB1126V
exceed length in 1KBH_B_YB1172W,IB1126V
exceed length in 1KBH_B_YB1172W,LB1131A
exceed length in 1KBH_B_YB1172W,LB1131A
exceed length in 1KBH_B_YB1172W,LB1134A
exceed length in 1KBH_B_YB1172W,LB1134A
exceed length in 1KBH_B_YB1172W,LB1138A
exceed length in 1KBH_B_YB1172W,LB1138A
exceed length in 1KBH_B_YB1172W,VB1150A
exceed length in 1KBH_B_YB1172W,VB1150A
exceed length in 1KBH_B_YB1172W,LB1151A
exceed length in 1KBH_B_YB1172W,LB1151A
exceed length in 1KBH_B_YB1172W,LB1160A
exceed length in 1KBH_B_YB1172W,LB1160A
exceed length in 1KBH_B_YB1172W,AB1163G
exceed length in 1KBH_B_YB1172W,AB1163G
exceed length in 1KBH_B_YB1172W,IB1165V
exceed length in 1KBH_B_YB1172W,IB1165V
exceed length in 1KBH_B_YB1172W,VB1173A
exceed length in 1KBH_B_YB1172W,VB1173A
exceed length in 1KBH_A_YB1172W,LA902A
exceed length in 1KBH_B_YB1172W,LA902A
exceed length in 1KBH_A_YB1172W,LA903A
exceed length i

1KBH is not that long
https://www.rcsb.org/structure/1KBH

In [45]:
seq_wild['1KBH_B']

'PNRSISPSALQDLLRTLKSPSSPQQQQQVLNILKSNPQLMAAFIKQRTAKYVANQPGMQ'

find the sequenc is 4CPA_I is one amino acids smaller, thus, change the position in csv file.

could be different version https://www.rcsb.org/sequence/4CPA#I

In [46]:
seq_wild['4CPA_I']

'HADPICNKPCKTHDDCSGAWFCQACWNSARTCGPYV'

In [47]:
len(seq_wild['4CPA_I'])

36

In [48]:
seq_wild['4CPA_I'].find('GPYV')

32

find the sequenc is 1S1Q is one amino acids smaller, thus, change the position in csv file

In [49]:
seq_wild['1S1Q_A'].find('VFND')

39

In [50]:
seq_wild['1S1Q_A']

'VSESQLKKVSKYKYRDLTVRETVNVITLYKDLKPVLDSYVFNDGSSRELNLTGTIPVPYRGNTYNIPICLWLLDTYPYNPPICFVKPTSSTIKTGKHVDANGKIYLPYLHEWKHPQSDLLGLIQVIVVFGDEPPVFS'

In [51]:
len(seq_mutated)

7107

In [52]:
f3 = open("./skempi/mut.seq.txt", "w")

for elem in seq_mutated:
    ent_name = elem
    seq = seq_mutated[ent_name]
    info = ent_name + " " + seq
    #print(info)
    print(info, file=open("./skempi/mut.seq.txt", "a"))

f3.close()